<a href="https://colab.research.google.com/github/Nekoiii/ML_Practices_colab/blob/main/Rola_test_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考文章：https://note.com/npaka/n/ndb287a48b682

In [27]:
%%script false --no-raise-error
from google.colab import drive
drive.mount('/content/drive')

In [17]:
%%script false --no-raise-error
#download Lora
%cd  /content/drive/MyDrive/ML_Practices/Lora_test_1
!pwd
!git clone https://github.com/cloneofsimo/lora.git
!pip install accelerate bitsandbytes

/content/drive/MyDrive/ML_Practices/Lora_test_1
/content/drive/MyDrive/ML_Practices/Lora_test_1
Cloning into 'lora'...
remote: Enumerating objects: 934, done.
remote: Counting objects: 100% (418/418), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 934 (delta 356), reused 307 (delta 307), pack-reused 516
Receiving objects: 100% (934/934), 182.96 MiB | 18.65 MiB/s, done.
Resolving deltas: 100% (561/561), done.
Updating files: 100% (60/60), done.


In [ ]:
%cd lora
!pip install .
!pip install -r requirements.txt
%cd ../
!pwd

In [ ]:
!git clone https://github.com/huggingface/transformers
%cd transformers
!pip install .
%cd ../

参考文章：https://torch.classcat.com/2023/04/12/blog-sd-webui-colab-lora-training-by-diffusers/

In [49]:
!pip install diffusers==0.14.0 transformers accelerate safetensors

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [55]:
!git clone https://github.com/huggingface/diffusers.git -b v0.14.0

Cloning into 'diffusers'...
remote: Enumerating objects: 26921, done.
remote: Counting objects: 100% (493/493), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 26921 (delta 286), reused 400 (delta 240), pack-reused 26428
Receiving objects: 100% (26921/26921), 18.37 MiB | 15.67 MiB/s, done.
Resolving deltas: 100% (19599/19599), done.
Updating files: 100% (752/752), done.


In [57]:
from accelerate.utils import write_basic_config

write_basic_config(mixed_precision="fp16", save_location="./lora/default_config.yaml")
!cat ./lora/default_config.yaml

{
  "compute_environment": "LOCAL_MACHINE",
  "distributed_type": "NO",
  "downcast_bf16": false,
  "machine_rank": 0,
  "main_training_function": "main",
  "mixed_precision": "fp16",
  "num_machines": 1,
  "num_processes": 1,
  "rdzv_backend": "static",
  "same_network": false,
  "tpu_use_cluster": false,
  "tpu_use_sudo": false,
  "use_cpu": false
}


In [ ]:
!pip install -r /content/drive/MyDrive/ML_Practices/Lora_test_1/diffusers/examples/dreambooth/requirements.txt

In [61]:
%%time

!accelerate launch --config_file="./lora/default_config.yaml" \
  /content/drive/MyDrive/ML_Practices/Lora_test_1/diffusers/examples/dreambooth/train_dreambooth_lora.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5"  \
  --instance_data_dir="./resized_sample_data_3" \
  --output_dir="./lora/output" \
  --instance_prompt="shs" \
  --resolution=512 \
  --train_batch_size=1 \
  --sample_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --checkpointing_steps=200 \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=3000 \
  --seed="0"

2023-06-08 12:56:46.712596: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[12:56:48] WARNING  The following values were not passed to        ]8;id=921872;file:///usr/local/lib/python3.10/dist-packages/accelerate/commands/launch.py\launch.py]8;;\:]8;id=442211;file:///usr/local/lib/python3.10/dist-packages/accelerate/commands/launch.py#913\913]8;;\
                    `accelerate launch` and had defaults used                   
                    instead:                                                    
                            `--num_processes` was set to a value                
                    of `1`                                                      
                            `--num_machines` was set to a value of              
                    `1`                                                         
                            `--mixed_precision` was set to a value              
                    of `'n

In [35]:
import torch
from diffusers import StableDiffusionPipeline, UniPCMultistepScheduler

pipe = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float16,
    ).to('cuda')

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [36]:
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows * cols

    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid

In [37]:
pipe.unet.load_attn_procs("/content/lora/output/pytorch_lora_weights.bin")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/diffusers/models/modeling_utils.py:857 in                │
│ _get_model_file                                                                                  │
│                                                                                                  │
│   854 │   │   │   │   )                                                                          │
│   855 │   │   try:                                                                               │
│   856 │   │   │   # 2. Load model file as usual                                                  │
│ ❱ 857 │   │   │   model_file = hf_hub_download(                                                  │
│   858 │   │   │   │   pretrained_model_name_or_path,                                             │
│   859 │   │   │   │   filename=weights_name,                                                     │
│   860 │   │   │   │   cache_dir=cache_dir,                                                       │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py:110 in _inner_fn    │
│                                                                                                  │
│   107 │   │   │   kwargs.items(),  # Kwargs values                                               │
│   108 │   │   ):                                                                                 │
│   109 │   │   │   if arg_name in ["repo_id", "from_id", "to_id"]:                                │
│ ❱ 110 │   │   │   │   validate_repo_id(arg_value)                                                │
│   111 │   │   │                                                                                  │
│   112 │   │   │   elif arg_name == "token" and arg_value is not None:                            │
│   113 │   │   │   │   has_token = True                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py:158 in              │
│ validate_repo_id                                                                                 │
│                                                                                                  │
│   155 │   │   raise HFValidationError(f"Repo id must be a string, not {type(repo_id)}: '{repo_   │
│   156 │                                                                                          │
│   157 │   if repo_id.count("/") > 1:                                                             │
│ ❱ 158 │   │   raise HFValidationError(                                                           │
│   159 │   │   │   "Repo id must be in the form 'repo_name' or 'namespace/repo_name':"            │
│   160 │   │   │   f" '{repo_id}'. Use `repo_type` argument if needed."                           │
│   161 │   │   )                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': 
'/content/lora/output/pytorch_lora_weights.bin'. Use `repo_type` argument if needed.

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/diffusers/loaders.py:147 in load_attn_procs              │
│                     

In [ ]:
#generator = [torch.Generator(device="cpu").manual_seed(i) for i in range(4)]

images = pipe(prompt=" (((shs))),1 girl,cute ,simple,happy,computer,business,white background,pure background,sticker",
              negative_prompt="authentic,real people,complicated,grey",
              #generator=generator,
              num_inference_steps=30,
              height=512,
              width=512,
              num_images_per_prompt=4,
              guidance_scale=7.0).images

image_grid(images, 2, 2)